In [104]:
import warnings
import pandas as pd
import sqlite3

warnings.filterwarnings('ignore')

In [105]:
def get_sub_totaled_dataframe(
    table_name: str, column_name: str, fiscal_year: str, business_unit_id: str
) -> pd.DataFrame() | None:
    conn = sqlite3.connect("main.db")

    query = """
        SELECT * FROM Budget_Account WHERE IsTotalAccount = 0 AND IsActiveTemplate = 1 
    """
    ba = pd.read_sql(query, conn)

    # first get the accounts
    query = f"""
        SELECT * 
        FROM {table_name} 
    """
    accounts = pd.read_sql(
        query, conn, dtype={"AccountNo": str, "FiscalYear": str, "BusinessUnitId": str}
    )
    accounts["AccountNo"] = accounts["AccountNo"].fillna("").astype(str)
    accounts["FiscalYear"] = accounts["FiscalYear"].fillna("").astype(str)
    accounts["BusinessUnitId"] = accounts["BusinessUnitId"].fillna("").astype(str)
    accounts["BusinessUnitId"]  = accounts["BusinessUnitId"].str.replace('.0', '')
    accounts = accounts[
        (accounts["FiscalYear"] == fiscal_year)
        & (accounts["BusinessUnitId"] == business_unit_id)
    ]
    account_group = accounts.groupby(["AccountNo"])["Amount"].sum().reset_index()
    accounts = pd.merge(ba, account_group, on="AccountNo", how="left")
    accounts = accounts[~pd.notna(accounts["RAD"])]

    # next get the rads
    query = f"""
        SELECT * 
        FROM {table_name} master 
        JOIN {table_name}_Rad master_rad ON master_rad.{table_name}Id = master.Id
        JOIN RAD r ON r.RadId = master_rad.RADID
    """
    rads = pd.read_sql(query, conn)
    rads = rads[
        (rads["FiscalYear"] == fiscal_year)
        & (rads["BusinessUnitId"] == business_unit_id)
    ]
    rads_group = rads.groupby(["RAD"])["Amount"].sum().reset_index()
    rads = pd.merge(ba, rads_group, on="RAD", how="left")
    rads = rads.dropna(subset="RAD")

    # Filter to get only AccountNo with two or more entries
    subtotal = rads.groupby(["AccountNo", "DisplayOrder"])["Amount"].sum().reset_index()
    account_counts = rads["AccountNo"].value_counts()
    valid_accounts = account_counts[account_counts >= 2].index

    # Get subtotal for valid rows that have a two or more RADs
    subtotal_filtered = subtotal[subtotal["AccountNo"].isin(valid_accounts)]
    max_display_order = (
        subtotal.groupby("AccountNo")["DisplayOrder"].max().reset_index()
    )
    subtotal_filtered = subtotal_filtered.merge(
        max_display_order, on="AccountNo", how="left", suffixes=("", "_max")
    )
    subtotal_filtered = subtotal_filtered.reset_index()
    subtotal_filtered["AccountNo"] = (
        subtotal_filtered["AccountNo"].astype(str) + " SubTotal"
    )
    subtotal_filtered = subtotal_filtered.groupby(["AccountNo", "DisplayOrder_max"])[
        "Amount"
    ].sum()
    subtotal_filtered = subtotal_filtered.reset_index()
    subtotal_filtered['IsSubTotal'] = 1

    # set display order of subtotal row
    subtotal_filtered["DisplayOrder_max"] = subtotal_filtered["DisplayOrder_max"] + 0.1
    subtotal_filtered.rename(columns={"DisplayOrder_max": "DisplayOrder"}, inplace=True)

    # combine rad and subtotal dataframe
    sub_total_rads = pd.concat([subtotal_filtered, rads])
    sub_total_rads = sub_total_rads.sort_values(by="DisplayOrder")

    # finally combine with the accounts dataframe
    master = pd.concat([sub_total_rads, accounts])
    master = master.sort_values(by="DisplayOrder")
    master = master[["IsSubTotal", "DisplayOrder", "AccountNo", "Account", "RAD", "Amount"]]
    master['IsSubTotal'] = master['IsSubTotal'].fillna(0)
    master.rename(columns={"Amount": f"{column_name}Total"}, inplace=True)

    return master

In [106]:
kwargs = {
    "table_name": "Budget",
    "column_name": "Budgets",
    "fiscal_year": "FY24",
    "business_unit_id": "9500",
}
budgets = get_sub_totaled_dataframe(**kwargs)
budgets.head()

,IsSubTotal,DisplayOrder,AccountNo,Account,RAD,BudgetsTotal
0,0.0,1.0,50000,Salaries,Regular,NaN
1,0.0,2.0,50000,Salaries,Overtime,NaN
0,1.0,2.1,50000 SubTotal,NaN,NaN,0.0
2,0.0,3.0,51000,Contracted Labor,None,150000.0
3,0.0,4.0,52500,Employee Relations,None,920.0


In [107]:
kwargs = {
    "table_name": "JournalEntry",
    "column_name": "Actuals",
    "fiscal_year": "FY24",
    "business_unit_id": "9500",
}
actuals = get_sub_totaled_dataframe(**kwargs)
actuals.head()

,IsSubTotal,DisplayOrder,AccountNo,Account,RAD,ActualsTotal
0,0.0,1.0,50000,Salaries,Regular,1717544.42
1,0.0,2.0,50000,Salaries,Overtime,12391.28
0,1.0,2.1,50000 SubTotal,NaN,NaN,1729935.70
2,0.0,3.0,51000,Contracted Labor,None,77609.19
3,0.0,4.0,52500,Employee Relations,None,1198.03


In [108]:
master = pd.merge(actuals, budgets, on="DisplayOrder", how="left")
master = master[
    [
        "IsSubTotal_x",
        "DisplayOrder",
        "AccountNo_x",
        "Account_x",
        "RAD_x",
        "ActualsTotal",
        "BudgetsTotal",
    ]
]
master.columns = master.columns.str.replace("_x", "", regex=False)
master['Variance'] = master['BudgetsTotal'] - master['ActualsTotal']
master

,IsSubTotal,DisplayOrder,AccountNo,Account,RAD,ActualsTotal,BudgetsTotal,Variance
0,0.0,1.0,50000,Salaries,Regular,1717544.42,NaN,NaN
1,0.0,2.0,50000,Salaries,Overtime,12391.28,NaN,NaN
2,1.0,2.1,50000 SubTotal,NaN,NaN,1729935.70,0.0,-1729935.70
3,0.0,3.0,51000,Contracted Labor,None,77609.19,150000.0,72390.81
4,0.0,4.0,52500,Employee Relations,None,1198.03,920.0,-278.03
...,...,...,...,...,...,...,...,...
68,0.0,63.0,75000,Professional Development,Training and Seminar,9912.19,NaN,NaN
69,0.0,64.0,75000,Professional Development,No Prof Development,NaN,NaN,NaN
70,1.0,64.1,75000 SubTotal,NaN,NaN,13079.36,0.0,-13079.36
71,0.0,65.0,76000,Sponsorship/Benefaction,None,NaN,NaN,NaN


In [112]:
business_unit_id = "9500"
fiscal_year = "FY25"
query = f"SELECT * FROM ProposedBudget WHERE FiscalYear = '{fiscal_year}' AND BusinessUnitId = '{business_unit_id}';"
conn = sqlite3.connect("main.db")
proposed_budget = pd.read_sql(query, conn)
cols = [
    "Id",
    "AccountNo",
    "RAD",
    "ProposedBudget",
    "BusinessCaseName",
    "BusinessCaseAmount",
    "TotalBudget",
    "Comments",
    "IsSubTotal"
]
proposed_budget.rename(columns={'Id': 'ProposedBudgetId'}, inplace=True)
proposed_budget.head()

,ProposedBudgetId,FiscalYear,BusinessUnitId,AccountNo,RAD,ProposedBudget,BusinessCaseName,BusinessCaseAmount,TotalBudget,Comments,IsSubTotal
0,21,FY25,9500,50000,Regular,68.262304,9500,0.942189,69.204493,Hello World!,0
1,22,FY25,9500,50000,Overtime,84.803573,9500,2.054405,86.857978,Hello World!,0
2,23,FY25,9500,50000 SubTotal,None,7.246501,9500,12.105782,19.352282,Hello World!,1
3,24,FY25,9500,51000,None,72.666761,9500,32.408313,105.075074,Hello World!,0
4,25,FY25,9500,52500,None,56.753167,9500,30.910065,87.663232,Hello World!,0


In [114]:
merge = pd.merge(master, proposed_budget, on=['AccountNo', 'RAD', 'IsSubTotal'], how='left')
merge

,IsSubTotal,DisplayOrder,AccountNo,Account,RAD,ActualsTotal,BudgetsTotal,Variance,ProposedBudgetId,FiscalYear,BusinessUnitId,ProposedBudget,BusinessCaseName,BusinessCaseAmount,TotalBudget,Comments
0,0.0,1.0,50000,Salaries,Regular,1717544.42,NaN,NaN,21,FY25,9500,68.262304,9500,0.942189,69.204493,Hello World!
1,0.0,2.0,50000,Salaries,Overtime,12391.28,NaN,NaN,22,FY25,9500,84.803573,9500,2.054405,86.857978,Hello World!
2,1.0,2.1,50000 SubTotal,NaN,NaN,1729935.70,0.0,-1729935.70,23,FY25,9500,7.246501,9500,12.105782,19.352282,Hello World!
3,0.0,3.0,51000,Contracted Labor,None,77609.19,150000.0,72390.81,24,FY25,9500,72.666761,9500,32.408313,105.075074,Hello World!
4,0.0,4.0,52500,Employee Relations,None,1198.03,920.0,-278.03,25,FY25,9500,56.753167,9500,30.910065,87.663232,Hello World!
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,0.0,63.0,75000,Professional Development,Training and Seminar,9912.19,NaN,NaN,89,FY25,9500,0.544572,9500,31.062500,31.607072,Hello World!
69,0.0,64.0,75000,Professional Development,No Prof Development,NaN,NaN,NaN,90,FY25,9500,95.514306,9500,24.164471,119.678778,Hello World!
70,1.0,64.1,75000 SubTotal,NaN,NaN,13079.36,0.0,-13079.36,91,FY25,9500,70.454048,9500,6.696915,77.150963,Hello World!
71,0.0,65.0,76000,Sponsorship/Benefaction,None,NaN,NaN,NaN,92,FY25,9500,10.967345,9500,16.585979,27.553323,Hello World!
